# Binary Phase Field Benchmark

FiPy implementation of phase transormation in 2D with solutal driving force

**Do not edit `binary_phase_field.py`**. Generate the batch-runnable file from the notebook with
```bash
jupyter nbconvert binary_phase_field.ipynb --to python --output-dir=../scripts/
```

## Import Python modules

In [ ]:
import argparse
import json
import os
import re
import sys

Jupyter notebook handles some things differently than from the commandline

In [ ]:
isnotebook = False
try:
    from IPython import get_ipython
    isnotebook = (get_ipython().__class__.__name__ == "ZMQInteractiveShell")
except:
    pass

## Initialize
### Load parameters

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--solve_log", help="file to log solver status in",
                    default=None)
parser.add_argument("--restart", help="solution to initialize from",
                    default=None)
parser.add_argument("--checkpoint_interval", help="frequency to save results",
                    type=float, default=6.)
parser.add_argument("--totaltime", help="duration of full simulation",
                    type=float, default=600.)
parser.add_argument("--numberOfElements", help="number of total cells in a Grid2D",
                    type=int, default=1000000)
parser.add_argument("--solver", help="solver class to use", default="pcg")
parser.add_argument("--preconditioner", help="preconditioner class to use", default="none")
parser.add_argument("--sweeps", help="number of nonlinear sweeps to take",
                    type=int, default=5)
parser.add_argument("--iterations", help="maximum number of linear iterations to take for each sweep",
                    type=int, default=1000)
parser.add_argument("--tolerance", help="linear solver tolerance",
                    default=1e-10)
parser.add_argument("--store_matrix",
                    help="store the matrix and RHS vector along with other output",
                    action='store_true')
parser.add_argument("--gradient2thermal",
                    help="strength of gradient energy relative to thermal energy",
                    type=float, default=1)
parser.add_argument("--segregation2transformation",
                    help="strength of segregation relative to transformation driving force",
                    type=float, default=1)
parser.add_argument("--adsorption",
                    help="strength of relative adsorption",
                    type=float, default=0)
parser.add_argument("--profile",
                    help="store profiling statistics along with other output",
                    action='store_true')
parser.add_argument("--view",
                    help="whether to view the results",
                    action='store_true')

### Set any parameters for interactive notebook

In [101]:
if isnotebook:
    # argv = ["--numberOfElements=10000", "--totaltime=1.2", "--checkpoint_interval=0.12",
    #         "--nucleation_scale=100", "--solve_log=nucleation6/solver.log"]
    argv = ["--numberOfElements=100000", "--nucleation_scale=1000", "--solve_log=solidification3/solver.log",
            "--segregation2transformation=6.",
           "--restart=../../results/nucleation/nucleation18/t=300.0.npz",
           "--store_matrix"]
else:
    argv = None

In [102]:
args, unknowns = parser.parse_known_args(args=argv)

In [ ]:
if args.tolerance != "default":
    args.tolerance = float(args.tolerance)

### Setup ouput storage

In [ ]:
if (args.solve_log is not None):
    output = os.path.dirname(args.solve_log)
    os.makedirs(output, exist_ok=True)
else:
    output = os.getcwd()

### Setup logging

In [ ]:
if "FIPY_LOG_CONFIG" not in os.environ:
    log_config = os.path.join(output, "logger.json")

    with open(args.solve, "w") as f:
        f.write("""
{{
    "version": 1,
    "formatters": {{
        "default": {{
            "format": "%(asctime)s | %(levelname)s | %(name)s | %(funcName)s | %(message)s"
        }}
    }},
    "handlers": {{
        "serialfile": {{
            "class": "logging.FileHandler",
            "formatter": "default",
            "filename": "{solve_log}"
        }}
    }},
    "loggers": {{
        "fipy": {{
            "level": "DEBUG",
            "handlers": ["serialfile"]
        }}
    }}
}}
""".format(solve_log=args.solve_log))

    os.environ["FIPY_LOG_CONFIG"] = log_config

### Import FiPy

In [ ]:
%conda info

In [ ]:
import fipy as fp
from fipy.tools import numerix as nmx
from fipy.tools import parallelComm

### Initialize mesh and solution variables

Either restart from some `path/to/restart/t={time}.npz`, where the time is assigned to `elapsed`

or

Create a mesh based on parameters. Set
>  the computational domain is ... 1000×1000 

In [ ]:
nx = ny = int(nmx.sqrt(args.numberOfElements))
mesh = fp.Grid2D(nx=nx, ny=ny)
phi = fp.CellVariable(mesh=mesh, name=r"$\phi$", value=0., hasOld=True)
elapsed = 0.

In [ ]:
if args.restart is not None:
    data = nmx.load(args.restart)
    lower, upper = int((1000 - nx) / 2), int((1000 + nx) / 2)
    phi.setValue(data["phi"][lower:upper, lower:upper].flat)

    # scanf("%g") simulator
    # https://docs.python.org/3/library/re.html#simulating-scanf
    scanf_g = r"[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?"
    pattern = r".*t=({g})\.npz".format(g=scanf_g)
    elapsed = float(re.match(pattern, args.restart).group(1))

In [ ]:
x, y = mesh.cellCenters[0], mesh.cellCenters[1]
X, Y = mesh.faceCenters[0], mesh.faceCenters[1]

In [ ]:
C = fp.CellVariable(mesh=mesh, name="$C$", value=0.7 - 0.4 * phi, hasOld=True)

In [ ]:
if isnotebook and config["view"]:
    viewer = fp.Viewer(vars=C, datamin=0., datamax=1.)
    viewer.plot()

## Create solver

In [ ]:
precon = None

if args.preconditioner == "jacobi":
    precon = fp.JacobiPreconditioner()
elif args.preconditioner == "ilu":
    precon = fp.ILUPreconditioner()
elif args.preconditioner == "ssor":
    precon = fp.SSORPreconditioner()
elif args.preconditioner == "icc":
    precon = fp.ICPreconditioner()
elif args.preconditioner == "none":
    precon = None
else:
    precon = fp.solvers.__dict__[args.preconditioner]()

if args.solver == "cgs":
    solver_class = fp.LinearCGSSolver
elif args.solver == "gmres":
    solver_class = fp.LinearGMRESSolver
elif args.solver == "lu":
    solver_class = fp.LinearLUSolver
elif args.solver == "pcg":
    solver_class = fp.LinearPCGSolver
else:
    solver_class = fp.solvers.__dict__[args.solver]

In [ ]:
if solver_class == fp.LinearLUSolver:
    precon = None # preconditioned lu doesn't make any sense

In [ ]:
solver = solver_class(tolerance=config["tolerance"], criterion="RHS",
                      iterations=config["iterations"], precon=precon)

## Binary Alloy with Frozen Phase Field: governing equation

Given the strength of the phase transformation driving force,
$$\Delta f = \frac{1}{6\sqrt{2}}$$

In [ ]:
Delta_f = 1 / (6 * nmx.sqrt(2.))

the strength of gradient energy relative to thermal energy, $\xi$,

In [ ]:
xi = args.gradient2thermal

the strength of the segregation relative to the nucleation driving force, $\zeta$,

In [ ]:
zeta = args.segregation2transformation

the strength of the relative adsorption, $\tilde{W}$,

In [ ]:
Delta_W = args.adsorption

the phase interpolation function
\begin{align}
p(\phi) & \equiv \phi^3 (6\phi^2 - 15 \phi + 10)
\end{align}

In [ ]:
def p(phi):
    return phi**3 * (6 * phi**2 - 15 * phi + 10)

and the double well function
\begin{align}
g(\phi) & \equiv \phi^2 (1 - \phi)^2
\end{align}

In [ ]:
def g(phi):
    return phi**2 * (1 - phi)**2

diffusion with convection due to frozen phase boundary can be described by
\begin{align}
\frac{\partial C}{\partial t} &= \nabla^2 C
+ \nabla\cdot\left[
    C\left(1-C\right)
    \xi
    \left(
        \zeta \Delta f p'(\phi)
        - \frac{\Delta\tilde{W}}{2} g'(\phi)
    \right)
    \nabla \phi\right]
\\
&= \nabla^2 C
+ \nabla\cdot\left[
    C\left(1-C\right)
    \xi
    \left(
        \zeta \Delta f \nabla p(\phi)
        - \frac{\Delta\tilde{W}}{2} \nabla g(\phi)
    \right)
\right]
\end{align}

In [ ]:
phaseTransformationVelocity = (1 - C).harmonicFaceValue * xi * (zeta * Delta_f * p(phi).faceGrad 
                                                                - 0.5 * Delta_W * g(phi).faceGrad)

eq = (fp.TransientTerm(var=C)
      == fp.DiffusionTerm(var=C)
      + fp.ConvectionTerm(coeff=phaseTransformationVelocity, var=C))

## Free Energy

\begin{align}
\frac{f(\phi, C, T) V_m}{RT}
&= \xi \left\{ 
-\Delta f \left[1 + \frac{1}{2} (1 - 2C) \zeta
\right]p(\phi)\right.
\\
&\qquad \left. {} + \left[
1 - \frac{1}{2} (1 - 2C) \Delta\tilde{W}
\right] g(\phi)
\right\}
\\
&\qquad {} + \left[
    \left(1 - C\right)\ln\left(1 - C\right) + C\ln C
\right]
\end{align}

In [ ]:
def f(phi, C):
    return (xi * (-Delta_f * (1 + 0.5 * (1 - 2 * C) * zeta) * p(phi)
                  + (1 + 0.5 * (1 - 2 * C) * Delta_W) * g(phi))
            + (1 - C) * nmx.log(1 - C) + C * nmx.log(C))

In [ ]:
phi_C_mesh = fp.Grid2D(nx=100, Lx=1., ny=100, Ly=1.)
free_energy = f(phi=phi_C_mesh.y, C=phi_C_mesh.x)
free_energy.name = r"$\phi$ vs. $C$"
if isnotebook and config["view"]:
    fp.Viewer(vars=free_energy)

## Setup output

In [ ]:
if parallelComm.procID == 0:
    print("storing results in {0}".format(output))

### Define output routines

In [ ]:
def saveC(elapsed):
    C_value = C.globalValue.reshape((nx, ny))
    if parallelComm.procID == 0:
        fname = os.path.join(output, "t={}.npz".format(elapsed))
        nmx.savez(fname, C=C_value)

def saveMatrix(elapsed):
    mtxname = os.path.join(output, "t={}.mtx".format(elapsed))
    eq.matrix.exportMmf(mtxname)
    
    rhs_value = eq.RHSvector
    if parallelComm.procID == 0:
        rhsname = os.path.join(output, "t={}.rhs.npz".format(elapsed))
        nmx.savez(rhsname, rhs=rhs_value)

def save_energy():
    nx = phi_C_mesh.nx
    ny = phi_C_mesh.ny
    energy_value = free_energy.globalValue
    energy_value = energy_value.reshape((nx, ny))
    if parallelComm.procID == 0:
        fname = os.path.join(output, "energy.npz")
        nmx.savez(fname, energy=energy_value)
    
def checkpoint_data(elapsed, store_matrix=False):
    saveC(elapsed)
    if store_matrix:
        saveMatrix(elapsed)

In [ ]:
save_energy()

### Figure out when to save

In [ ]:
checkpoints = (fp.numerix.arange(int(elapsed / args.checkpoint_interval),
                                 int(args.totaltime / args.checkpoint_interval))
               + 1) * args.checkpoint_interval

checkpoints.sort()

## Solve and output

In [ ]:
times = checkpoints
times = times[(times > elapsed) & (times <= args.totaltime)]

In [ ]:
print(elapsed)

In [ ]:
print(times)

In [ ]:
print(config["totaltime"])

In [ ]:
if args.profile:
    import cProfile

    pr = cProfile.Profile()
    pr.enable()

In [ ]:
from steppyngstounes import CheckpointStepper, FixedStepper

eq.cacheMatrix()
eq.cacheRHSvector()

C.updateOld()
for checkpoint in CheckpointStepper(start=elapsed,
                                    stops=times,
                                    stop=args.totaltime):

    for step in FixedStepper(start=checkpoint.begin,
                             stop=checkpoint.end,
                             size=1.):

        state = dict(state="START", numberOfElements=mesh.numberOfCells, sweeps=args.sweeps)
        if precon is None:
            state["preconditioner"] = None
        else:
            state["preconditioner"] = precon.__class__.__name__

        solver._log.debug(json.dumps(state))

        for sweep in range(args.sweeps):
            res = eq.sweep(var=C, dt=step.size, solver=solver)

        state["state"] = "END"
        solver._log.debug(json.dumps(state))

        C.updateOld()
        # stats.append(current_stats(step.end))

        _ = step.succeeded(error=res / 1e-3)

    if checkpoint.end in checkpoints:
        # don't save nucleation events?
        checkpoint_data(checkpoint.end, store_matrix=args.store_matrix)

    if isnotebook and args.view:
        viewer.plot()
        # labelViewer.plot()

    _ = checkpoint.succeeded()

In [ ]:
if args.profile:
    pr.disable()
    
    pr.dump_stats(os.path.join(output, "profile-{0}.prof".format(parallelComm.procID)))